# Baseline Model

Here I am going to define a random guess model that is going to be the baseline against which

In [1]:
import pandas as pd 
import numpy as np

In [2]:
CTG = pd.read_csv("CTG.csv")
fs_labels = CTG['NSP']
mp_labels = CTG['CLASS']
data = CTG.drop(['CLASS', 'NSP'], axis = 1)

In [3]:
count = np.zeros(3)

for val in fs_labels:
    count[val - 1] += 1
        
real_class_freq = count/len(fs_labels)
for idx, freq in np.ndenumerate(real_class_freq):
    print "The frequency of class {0} is {1:.3}".format(idx[0]+1, freq)

The frequency of class 1 is 0.778
The frequency of class 2 is 0.139
The frequency of class 3 is 0.0828


In [4]:
# the value of the class of normal cases is set to -1, negative (absence of disease)
idxs = fs_labels[fs_labels == 1]
fs_labels[idxs.index] = -1

# suspicious and pathologic exams are merged in one class with positive value (presence of disease)
idxs = fs_labels[fs_labels == 2]
fs_labels[idxs.index] = +1

idxs = fs_labels[fs_labels == 3]
fs_labels[idxs.index] = +1


C:\Users\Ludovica\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Ludovica\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ludovica\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def random_data(length, prc):
    from random import shuffle
    
    data = np.ones(length)
    data[0 : round(prc*length)] = -1
    shuffle(data)
    
    return data

from sklearn.metrics import f1_score

rep = 1000000
f1_scores = 0
for i in range (0,rep):
    labels = random_data(len(fs_labels), real_class_freq[0])
    f1_scores += f1_score(fs_labels, labels, average='binary', pos_label=1)
    
baseline_score = f1_scores/rep
print "The baseline f1 score is {}".format(baseline_score)

C:\Users\Ludovica\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, fs_labels, test_size = 0.25)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier('stratified')

dummy_clf.fit(X_train,y_train)

pred = dummy_clf.predict(X_test)

print "The baseline f1 score is {}".format(f1_score(y_test, pred, average='binary', pos_label=1))